Data [Animal Image Dataset(DOG, CAT and PANDA)](https://www.kaggle.com/ashishsaxena2209/animal-image-datasetdog-cat-and-panda)<br>
Với 3000 hình, mỗi class 1000 hình

Tạo class SimplePreprocessor để xử lý hình ảnh. Trong này chỉ mới ở mức sử lý đơn giản là resize lại ảnh mà không quan tâm đến tỷ lệ gốc.<br>
Việc tạo class để quản lý dễ hơn các tác vụ trong mô hình.

**A Basic Image Preprocessor**

In [3]:
# import the necessary packages
import cv2

class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # store the target image width, height, and interpolation method used when resizing
        self.width = width
        self.height = height
        self.inter = inter
    
    def preprocess(self, image):
        # resize the image to a fixed size, ignoring the aspect ratio
        return cv2.resize(image, (self.width, self.height), interpolation= self.inter)

**Building an Image Loader**

In [4]:
# import the necessary packages
import numpy as np
import os
import cv2

class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        # store the image preprocessor
        self.preprocessors = preprocessors
        
        # if the preprocessors are None, initialize them as an empty list
        if self.preprocessors is None:
            self.preprocessors = []
    
    def load(self, imagePaths):
        # initialize the list of features and labels
        data = []
        labels = []
        # loop over the input images
        for root, dirs, files in os.walk(imagePaths, topdown=False):
            for name in files:
                image = cv2.imread(os.path.join(root, name))
                label = root.split("\\")[-1]
            
                # check to see if our preprocessors are not None
                if self.preprocessors is not None:
                    # loop over the preprocessors and apply each to the image
                    for p in self.preprocessors:
                        image = p.preprocess(image)
            
                # treat our processed image as a "feature vector"
                # by updating the data list followed by the label
                data.append(image)
                labels.append(label)
                
        # return a tuple of the data and labels
        return (np.array(data), np.array(labels))

- Các bước ở class này là check xem có sử dụng tiền xử lý hình ảnh hay không nếu không thì set là một empty list
- Phần load lấy đường dẫn của từng hình rồi đọc bằng cv2 qua `imread` , lấy label rồi add tuần tự các hình và nhãn vào **data** và **labels** sau đó trả về tuple mà mỗi phần tử là mảng numpy array
- Chú ý: Nếu có sử dụng tiền xử lý hình ảnh thì kiểm tra và xử lý, rồi cũng thêm vào **data** như trên

**k-NN: A Simple Classifier**

In [5]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [6]:
path = "D:\\CBD robotics course\\data\\animal-image-datasetdog-cat-and-panda\\animals"

In [7]:
# initialize the image preprocessor, load the dataset from disk,
# and reshape the data matrix
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(path)

- Sử dụng tiền xử lý `SimplePreprocessor` là resize kích cỡ về 32x32
- Load dữ liệu

**Hold out**

In [8]:
# encode the labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

Label encode các nhãn, chia train test với 75% train và 25% test

In [9]:
trainX.shape

(2250, 32, 32, 3)

Tập trainX và testX có 4 chiều. Trong khi ta "định" thử với mô hình KNN. Mô hình này chỉ nhận 2 chiều nên ta reshape lại 2 tập

In [10]:
trainX = trainX.reshape((trainX.shape[0], 32*32*3))
testX = testX.reshape((testX.shape[0], 32*32*3))

In [11]:
trainX.shape

(2250, 3072)

**Evaluate model**

In [12]:
model = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)
model.fit(trainX, trainY)
y_predicted = model.predict(testX)
print(classification_report(testY, y_predicted))

              precision    recall  f1-score   support

           0       0.41      0.49      0.45       262
           1       0.35      0.47      0.40       249
           2       0.70      0.32      0.44       239

    accuracy                           0.43       750
   macro avg       0.49      0.43      0.43       750
weighted avg       0.49      0.43      0.43       750



Kết luận: vậy là ta đã có được mô hình đơn giản để phân loại chó, mèo và gấu trúc mặc dù mô hình này rất tệ.

**Tune model**

In [17]:
from sklearn.model_selection import GridSearchCV

params = {'n_neighbors': [1,2,3,4,5],
         'weights': ['uniform', 'distance'],
         'n_jobs': [None, -1]}
grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=params, scoring='f1_micro', cv=5, n_jobs=-1)

In [18]:
grid.fit(trainX, trainY)
print("Best Model Parameter: ",grid.best_params_)

Best Model Parameter:  {'n_jobs': None, 'n_neighbors': 5, 'weights': 'distance'}


In [19]:
grid.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='distance')

In [21]:
y_predicted=  grid.predict(testX)
print(classification_report(testY, y_predicted))

              precision    recall  f1-score   support

           0       0.45      0.56      0.50       262
           1       0.38      0.49      0.43       249
           2       0.86      0.35      0.50       239

    accuracy                           0.47       750
   macro avg       0.56      0.47      0.48       750
weighted avg       0.56      0.47      0.48       750



Mô hình sau khi điều chỉnh hyperparameters thì kết quả đã được cải thiện. Dù vậy kết quả vẫn chưa tốt. Có thẻ kết luận dùng mô hình KNN để phân loại ảnh có hiệu quả thấp

In [22]:
encoded_labels = np.array([0,1,2])
le.inverse_transform(encoded_labels)

array(['cats', 'dogs', 'panda'], dtype='<U5')

Trong đó ta để ý thấy nhãn panda (sau khi encode là 2) có precision cao nhất. Có lẽ vì loài gấu trúc có hình dạng khá gần với chó và với mèo??